In [1]:
import gzip
import math
import random
from collections import defaultdict
import numpy as np



pathway = "C:/Users/brand/Desktop/goodreads_reviews_comics_graphic.json.gz"


def parseData(fname):
    for l in gzip.open(fname):
        d = eval(l)
        yield d


data = list(parseData(pathway))

usersPerItem = defaultdict(set)
itemsPerUser = defaultdict(set)
ratingDict = {}

for d in data:
    user, item = d['user_id'], d['book_id']
    usersPerItem[item].add(user)
    itemsPerUser[user].add(item)
    ratingDict[(user,item)] = d['rating']
    

In [2]:
print("Problem 1")
print()

def Jaccard(s1,s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    return numer/denom


def most_similar(i):
    sims = []
    for j in usersPerItem:
        if j == i:
            continue
        
        sim = Jaccard(usersPerItem[i], usersPerItem[j])
        sims.append((sim,j))
    
    sims.sort(reverse=True)
    return sims[0:10]

print(most_similar('18471619'))

Problem 1

[(0.16666666666666666, '25334626'), (0.14285714285714285, '25659811'), (0.13793103448275862, '18369278'), (0.13157894736842105, '18430205'), (0.12903225806451613, '20299669'), (0.125, '17995154'), (0.12121212121212122, '23241671'), (0.12121212121212122, '23093378'), (0.12121212121212122, '18853527'), (0.11764705882352941, '26778333')]


In [3]:
testrat = 'dc3763cdb9b2cae805882878eebb6a32'
tr_favorite = '18471619'
print('problem 2')
print()

def user_similar(guy):
    sims = []
    for j in itemsPerUser:
        if j == guy:
            continue
        
        sim = Jaccard(itemsPerUser[guy], itemsPerUser[j])
        sims.append((sim,j))
    
    sims.sort(reverse=True)
    return sims[0:11]


testbuddy = '6470c7f5e3468ba34e9fe628960fbbf1'

lst3 = list(itemsPerUser[testbuddy])
#print(lst3)

    
def user_fav(user):
    lst = list(itemsPerUser[user])
    recs = []
    
    for i in lst:
        (x,y) = (user, i)
        recs.append((x,y,ratingDict[x,y]))
    
    sort_spec = ((1,True),(2,False))
    for index, reverse_value in sort_spec[::-1]:
        recs.sort(key = lambda x: x[index], reverse=reverse_value)
    
    #recs = sorted(recs, reverse = True, key = lambda x: (x[2], x[1]))
    
    return recs

def similar_users_favitems(buddy):
    
    lst = []
    options = list(user_similar(buddy))
    
    for i in options:
        user = i[1]
        rec = user_fav(user)
        
        for j in rec:
            if j[1] not in itemsPerUser[buddy]:
                lst.append(j)
                break
    
    
    
    return lst

print("Items most similar to users favorite: (similarity measure, item_id)")
print(most_similar(tr_favorite))
print('-----------------')
print("Top ten most similar users to our guy")
print(user_similar(testrat))
print("-------------------")
print("Top 10 recommended items in a list of tuples: (userid, item id, rating)")
print("----------------")
print(similar_users_favitems(testrat))
print("-----------------")
print(len(similar_users_favitems(testrat)))

problem 2

Items most similar to users favorite: (similarity measure, item_id)
[(0.16666666666666666, '25334626'), (0.14285714285714285, '25659811'), (0.13793103448275862, '18369278'), (0.13157894736842105, '18430205'), (0.12903225806451613, '20299669'), (0.125, '17995154'), (0.12121212121212122, '23241671'), (0.12121212121212122, '23093378'), (0.12121212121212122, '18853527'), (0.11764705882352941, '26778333')]
-----------------
Top ten most similar users to our guy
[(1.0, '4ae069d704b11bdf12c25fe640f75ff0'), (0.3333333333333333, '6470c7f5e3468ba34e9fe628960fbbf1'), (0.25, '6497ca91df3c182006874c96a8530b37'), (0.2, '033cf640dfa6f85eb146c39787289628'), (0.14285714285714285, '5510684ab6c18f2dd493787e66b2722c'), (0.05555555555555555, '17f73ea38e97307935c0d3b6ca987b53'), (0.030303030303030304, 'a39b4249d201ef5ce5ea553bdd013e66'), (0.023809523809523808, '42519f961f79b61701bda60787b031cf'), (0.02040816326530612, '65a7975989734fc6e18b7d2bd2bcb49f'), (0.014925373134328358, '0fafb6f0843124383f

In [4]:
print('problem 3')
print()

def pearson(i1,i2):
    
    inter = usersPerItem[i1].intersection(usersPerItem[i2])
    numer = 0 
    denom1 = 0 
    denom2 = 0
    
    for j in inter:
        numer += ratingDict[(j,i1)] * ratingDict[(j,i2)]
    
    for j in inter:
        denom1 += (ratingDict[(j, i1)])**2
        denom2 += (ratingDict[j,i2])**2
    
    denom = math.sqrt(denom1) * math.sqrt(denom2)
    
    if denom == 0:
        return 0
    
    return numer / denom

def pearson_all_den(i1,i2):
    
    inter = usersPerItem[i1].intersection(usersPerItem[i2])
    numer = 0 
    denom1 = 0 
    denom2 = 0
    
    for j in inter:
        numer += ratingDict[(j,i1)] * ratingDict[(j,i2)]
    
    for j in usersPerItem[i1]:
        denom1 += (ratingDict[(j, i1)])**2
    
    for j in usersPerItem[i2]:
        denom2 += (ratingDict[(j,i2)])**2
    
    denom = math.sqrt(denom1) * math.sqrt(denom2)
    
    if denom == 0:
        return 0
    
    return numer / denom


def most_similar(item,N,function):
    sims = [] 
    users = usersPerItem[item]
    for j in usersPerItem:
        if j == item:
            continue
        
        sim = function(item,j)
        sims.append((sim,j))
    
    #sims = sorted(sims,reverse=True, key = lambda x: (x[0],x[1]))
    sort_spec = ((0,True),(1,False))
    for index, reverse_value in sort_spec[::-1]:
        sims.sort(key = lambda x: x[index], reverse=reverse_value)
    
    return sims[0:N]


#print(usersPerItem[tr_favorite])
print("Top 10 pearson similar items, intersection denominator sorted alphebetically: ")
print("----------------")
print(most_similar(tr_favorite,10,pearson)) 
print('-----------------')
print()
print()



print("Top 10 pearson similar items, |a||b| denominator sorted alphebetically: ")
print("----------------")
print(most_similar(tr_favorite,10,pearson_all_den))
print('-----------------')

problem 3

Top 10 pearson similar items, intersection denominator sorted alphebetically: 
----------------
[(1.0000000000000002, '1228646'), (1.0000000000000002, '12298865'), (1.0000000000000002, '13227319'), (1.0000000000000002, '13826628'), (1.0000000000000002, '14654402'), (1.0000000000000002, '15793709'), (1.0000000000000002, '17131953'), (1.0000000000000002, '17295620'), (1.0000000000000002, '17295628'), (1.0000000000000002, '17417216')]
-----------------


Top 10 pearson similar items, |a||b| denominator sorted alphebetically: 
----------------
[(0.3371708921694098, '28182906'), (0.3211820274187864, '25659811'), (0.3018889419011508, '21814146'), (0.30104543943697304, '25334626'), (0.2934695928267111, '25955201'), (0.29025095010195523, '17995154'), (0.28658350270229216, '20299669'), (0.2854716408418839, '32500068'), (0.28361065854928985, '26778333'), (0.2809757434745082, '18767262')]
-----------------


In [14]:
ratingMean = sum([d['rating'] for d in data]) / len(data)
reviewsPerUser = defaultdict(list)
reviewsPerItem = defaultdict(list)

for d in data:
    user,item = d['user_id'], d['book_id']
    reviewsPerUser[user].append(d)
    reviewsPerItem[item].append(d)

userAverages = {}
itemAverages = {}

for u in itemsPerUser:
    rs = [ratingDict[(u,i)] for i in itemsPerUser[u]]
    userAverages[u] = sum(rs) / len(rs)
    
for i in usersPerItem:
    rs = [ratingDict[(u,i)] for u in usersPerItem[i]]
    itemAverages[i] = sum(rs) / len(rs)

def predictRating(user,item):
    ratings = []
    similarities = []
    for d in reviewsPerUser[user]:
        i2 = d['book_id']
        if d['book_id'] == item:
            continue
            
        ratings.append(d['rating'] - itemAverages[i2])
        similarities.append(Jaccard(usersPerItem[item],usersPerItem[i2]))
        
    if (sum(similarities) > 0):
        weightedRatings = [(x*y) for x,y in zip(ratings,similarities)]
        return itemAverages[item] + sum(weightedRatings) / sum(similarities)
    
    else:
        # User hasn't rated any similar items
        return ratingMean


#print(predictRating(testrat,'25659811'))

def MSE(preds,labels):
    diffs = [(x-y)**2 for x,y in zip(preds,labels)]
    return sum(diffs)/len(diffs)

simpreds = [predictRating(d['user_id'], d['book_id']) for d in data]
trainlabels = [d['rating'] for d in data]

print("++++++++++++++++++++++++++++")
print()
print("MSE: " + str(MSE(simpreds,trainlabels)))
print()
print("+++++++++++++++++++++++++++++")

++++++++++++++++++++++++++++

MSE: 0.8097344461671737

+++++++++++++++++++++++++++++


In [46]:
"""Modify Rating Prediction to include fucntion of time where the temporal function is defined to be e^(-.1)|yeari-yearj|"""

weekday, month, day, time, something, year = map(str,data[0]['date_added'].split(" "))

#print(datetime.datetime(data[0]['date_added']) - date.datetime(data[1]['date_added']))
#print(month)
#print(reviewsPerUser[testrat])
def temporal_function(year1,year2,lamb):
    year1 = int(year1)
    year2 = int(year2)
    
    return math.exp(-1*lamb*(abs(year1 - year2)))

def predictRating_temporal(user,item,lamb):
    ratings = []
    similarities = []
    temp = []
    year = 0           
    for d in reviewsPerUser[user]:
        i2 = d['book_id']
        i3 = d['date_added']
        weekday3, month3, day3, time3, something3, year3 = map(str,i3.split(" "))
        if d['book_id'] == item:
            i4 = d['date_added']
            weekday, month, day, time, something, year = map(str,i4.split(" "))
            continue
             
        ratings.append(d['rating'] - itemAverages[i2])
        similarities.append(Jaccard(usersPerItem[item],usersPerItem[i2]))
        temp.append(temporal_function(year,year3,lamb))
        
    if (sum(similarities) > 0):
        weightedRatings = [(x*y*z) for x,y,z in zip(ratings,similarities,temp)]
        weightedDenom = [(x*y) for x,y in zip(similarities,temp)]
        return itemAverages[item] + sum(weightedRatings) / sum(weightedDenom)
    
    else:
        # User hasn't rated any similar items
        return ratingMean

#print(temporal_function(2015,2021,.1))
preds_part = [predictRating_temporal(d['user_id'], d['book_id'],.1) for d in data]
partlabels = [d['rating'] for d in partitioned_data]

print(MSE(preds_part,partlabels))

0.7438108335753034
